In [ ]:
!pip install azure-storage-blob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 383 kB 13.6 MB/s 
     |████████████████████████████████| 4.1 MB 36.1 MB/s 
     |████████████████████████████████| 172 kB 46.6 MB/s 
     |████████████████████████████████| 85 kB 2.0 MB/s 
     |████████████████████████████████| 41 kB 293 kB/s 


In [ ]:
# 필요한 패키지 불러오기

import os, uuid # 파일 다운로드, uuid:임시변수
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# Blob 서비스에 연결하기

connect_string = 'DefaultEndpointsProtocol=https;AccountName=labuser3storage2;AccountKey=wHDUj4foRnixGsYyzwteay2zKB0Dy0hhjQq6CWg4AznFcONKlkAxTwsBiVViAOBKeN8v0n7zuHoA+AStkQJGcQ==;EndpointSuffix=core.windows.net'# 접근할 연결 문자열

blob_service_client = BlobServiceClient.from_connection_string(connect_string) # Blob 서비스에 연결 한 클라이언트 생성됨

In [ ]:
# Container 생성하기

# 컨테이너 이름 매번 만들수는 없다.
container_name = str(uuid.uuid4()) # 겹치지 않는 난수 숫자열 생성
print(container_name)

#실제 컨테이너 만드는 역할
container_client =  blob_service_client.create_container(container_name) # 만든 컨테이너 이름으로 컨테이너 생성해주기 

96bb1ec7-41e6-4ceb-8fb6-3d73eab53876


In [ ]:
# Container에 데이터 업로드하기

# 경로 지정
local_path = './data' # Data 폴더 아래
os.mkdir(local_path)  # Data 폴더 만들기

local_file_name = str(uuid.uuid4()) + '.txt'  # 텍스트 파일
upload_file_path = os.path.join(local_path, local_file_name) # 경로 합치기

In [ ]:
# 업로드 할 파일 준비하기

file = open(file=upload_file_path, mode='w') # 파일 쓰기모드로 새로 열기
file.write('Hello Azure Storage')
file.close # 꼭 닫아주기

# 새로운 blob client 만들기 : upload 공간 확보
blob_client = blob_service_client.get_blob_client(
    container=container_name,
    blob=local_file_name) # 업로드 파일 정보(경로)

# Blob에 파일 업로드하기
with open(file=upload_file_path, mode='rb') as data:   #binary 형태로 읽기 why? 파일이 깨질 수 가 있기 때문에
  blob_client.upload_blob(data)

In [ ]:
# Blob 안에 있는 파일의 목록 확인 open상태 : conainer
blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t' + blob.name)

	fccb713f-bdc6-4842-aa86-b4678b7d74b4.txt


In [ ]:
# upload된 파일 다운로드

download_file_path = os.path.join(local_path, str.replace(local_file_name, '.txt', 'DOWNLOAD.txt'))

# container_client 객체가 실제로 파일을 다운로드
container_client = blob_service_client.get_container_client(container=container_name) # upload파일과 download 파일이 다를 수 있으니 체크

# str으로 날라오면 이것을 코드로 보여줘야한다
with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall()) # container client한테 전체 데이터 다운받도록하기

In [ ]:
# 실습한 자원 정리하기

print('Press the Enter key to begin clean up')
input()

print('Deletint blob container...')
container_client.delete_container()

print('Deleting the local source and downloaded files...')# 다운받은 파일 정리
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)

print('Done')